# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.  We will use the Cart-Pole swing-up task as an example, and we will tune an MPC pipeline consisting of a multi-layer perception (MLP) system ID model and an iLQR optimizer.

## Set-Up

To begin, we need an input dataset and specifications for the system and task.  The `benchmarks` module provides these for a few example systems.

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

Loading AutoMPC...
Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed
Finished loading AutoMPC
running build_ext


Next, we need to declare our MPC pipeline.  The following code initializes a pipeline with a MLP system ID model, a quadratic objective function, and and an iLQR optimizer.

In [2]:
from autompc.sysid import MLPFactory
from autompc.control import IterativeLQRFactory
from autompc.costs import QuadCostFactory

model_factory = MLPFactory(system)
ctrlr_factory = IterativeLQRFactory(system)
cost_factory  = QuadCostFactory(system)

pipeline = ampc.Pipeline(system, model_factory, ctrlr_factory, cost_factory)

We can view the joint configuration space of the pipeline

In [3]:
pipeline.get_configuration_space()

Configuration space object:
  Hyperparameters:
    _cost:dx_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:dx_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:omega_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:theta_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:u_R, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_F, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _cost:x_Q, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    _ctrlr:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    _model:hidden_size_1, Type: UniformInteger, Range: [16, 256], Default: 128
   

## Tuning

Tune with selected surrogate model class, default model configuration.

In [4]:
from autompc.tuning import PipelineTuner

surrogate_factory = MLPFactory(system)
tuner = PipelineTuner(surrogate_mode="defaultcfg", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100))

/usr/lib/python3/dist-packages/pyparsing.py:1745: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:23<00:00,  1.67s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-29_19:41:20_790610
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:40<00:00,  2.00s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:11<00:00, 16.55it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 200.0000


...


Surrogate Cost:  200.0
Surrogate Final State:  [3.07377729 0.03142495 2.93310611 0.69010844]
use_cuda=True
MLP Using Cuda
hidden_sizes= [238, 128]
100%|██████████| 50/50 [01:25<00:00,  1.72s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:18<00:00, 10.90it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 104.038280 sec, available: 0.000010 sec)



Surrogate Cost:  63.0
Surrogate Final State:  [-0.00941103 -0.00449236 -0.13345293  0.00105161]
use_cuda=True
MLP Using Cuda
hidden_sizes= [255, 128]
100%|██████████| 50/50 [01:27<00:00,  1.74s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:19<00:00, 10.37it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 106.163151 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 4
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 12363.24 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 12229.17 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Final Incumbent: Configuration:
  _cost:dx_F, Value: 33.6356525523436
  _cost:dx_Q, Value: 0.0013622084435760135
  _cost:omega_F, Value: 1.3519710652415837
  _cost:omega_Q, Value: 0.01869440877657144



Surrogate Cost:  74.0
Surrogate Final State:  [-0.00711933  0.0564036  -0.14651211  0.08580299]
use_cuda=True
MLP Using Cuda
hidden_sizes= [238, 128]
100%|██████████| 50/50 [01:26<00:00,  1.74s/it]


We can also set the configuration of the surrogate model.

In [ ]:
surrogate_cfg = surrogate_factory.get_configuration_space().get_default_configuration()
surrogate_cfg["n_hidden_layers"] = "2"
surrogate_cfg["hidden_size_1"] = 128
surrogate_cfg["hidden_size_2"] = 128       
surrogate_cfg["nonlintype"] = "relu"

tuner = PipelineTuner(surrogate_mode="fixedcfg", surrogate_factory=surrogate_factory,
                     surrogate_cfg=surrogate_cfg, surrogate_split=0.5)

controller, tune_result2 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

/home/william/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


use_cuda=True
MLP Using Cuda
hidden_sizes= [128, 128]
100%|██████████| 50/50 [01:25<00:00,  1.71s/it]

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-29_23:10:17_436873
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 25 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent



use_cuda=True
MLP Using Cuda
hidden_sizes= [136, 136, 136]
100%|██████████| 50/50 [01:46<00:00,  2.12s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:12<00:00, 15.95it/s]
Surrogate Cost:  200.0
Surrogate Final State:  [ 3.16447529e+00 -1.86618074e-03  4.48800437e-02 -3.67070618e-02]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:12<00:00, 16.06it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 200.0000


...


True Dynamics Cost:  200.0
True Dynamics Final State:  [ 1.27489512 -0.45902782  2.06673456  0.56921877]
use_cuda=True
MLP Using Cuda
hidden_sizes= [43]
100%|██████████| 50/50 [01:00<00:00,  1.21s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:09<00:00, 20.45it/s]
Surrogate Cost:  200.0
Surrogate Final State:  [ 3.16950186 -0.00642914  0.1344617  -0.03765607]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:09<00:00, 20.90it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 79.628985 sec, available: 0.000010 sec)



True Dynamics Cost:  200.0
True Dynamics Final State:  [ 3.14309995e+00 -4.49769940e-03  7.73832516e-02 -5.66327064e-05]
use_cuda=True
MLP Using Cuda
hidden_sizes= [46, 128, 131, 128]
100%|██████████| 50/50 [02:16<00:00,  2.74s/it]
Simulating Surrogate Trajectory: 
 99%|█████████▉| 198/200 [00:57<00:00,  3.43it/s]
Surrogate Cost:  200.0
Surrogate Final State:  [ 0.0045206   0.02229831 -0.59331264  0.19524693]
Simulating True Dynamics Trajectory
 99%|█████████▉| 198/200 [00:53<00:00,  3.70it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 248.087276 sec, available: 0.000010 sec)



True Dynamics Cost:  200.0
True Dynamics Final State:  [ 0.2341417  -0.58883576 -0.32106375 -2.43814953]
use_cuda=True
MLP Using Cuda
hidden_sizes= [53, 144, 75]
  4%|▍         | 2/50 [00:04<01:39,  2.07s/it]

We can also auto-tune a particular surrogate class

In [ ]:
tuner = PipelineTuner(surrogate_mode="autotune", surrogate_factory=surrogate_factory, surrogate_split=0.5)

controller, tune_result3 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

Finally, we can auto-select and auto-tune the surrogate mode class

In [ ]:
tuner = PipelineTuner(surrogate_mode="autoselect", surrogate_split=0.5)

controller, tune_result4 = tuner.run(pipeline, task, trajs, n_iters=100, rng=np.random.default_rng(100),
                                    truedyn=benchmark.dynamics)

Finally, we can pre-train and select our own surrogate.

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

sysid_trajs = trajs[:250]
surr_trajs = trajs[250:]

model_evaluator = HoldoutModelEvaluator(system=system,trajs=surr_trajs,rng=np.random.default_rng(100),
                                        holdout_prop=0.25, metric="rmse")
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(surrogate_factory)

surrogate_model, _ = model_tuner.run(rng=np.random.default_rng(100), n_iters = 1)

tuner = PipelineTuner(surrogate_mode="pretrain")

controller, tune_result5 = tuner.run(pipeline, task, sysid_trajs, n_iters=100, surrogate=surrogate_model,
                                    truedyn=benchmark.dynamics)

## Decoupled Tuning

The above examples is full pipeline tuning, which searches the configuration space of all components simultaneously.  Alternatively, we can take a decoupled tuning approach, where the model is first tuned based on prediction accuracy, then the objective function and optimizer are tuned.

First, we must tune the model

In [ ]:
from autompc.tuning import ModelTuner
from autompc.evaluation import HoldoutModelEvaluator

model_evaluator = HoldoutModelEvaluator(holdout_prop=0.25, metric="rmse", trajs=trajs, system=system,
                                       rng=np.random.default_rng(100))
model_tuner = ModelTuner(system, model_evaluator)
model_tuner.add_model_factory(model_factory)

model, model_tune_result = model_tuner.run(rng=np.random.default_rng(100), n_iters=2)

Now that we have tuned the model, we can create a pipeline with the pre-tuned model, and then run pipeline tuning.

In [ ]:
pipeline_fixed_model = ampc.Pipeline(system, model, iLQRFactory, QuadCostFactory)
controller6, tune_result6 = tuner.run(pipeline, task, trajs, n_iters=100, truedyn=benchmark.dynamics)   